# DiaGuide LLM Analysis

## Comparative Evaluation of Large Language Models and Healthcare Professionals in Diabetes Guidance

This notebook reproduces the results and figures from the study comparing GPT-4o responses with healthcare professional responses to diabetes-related questions.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from scipy.stats import chi2_contingency, mannwhitneyu, kruskal
import warnings
warnings.filterwarnings('ignore')

# Set the style for the plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12
})


## 1. Load the dataset


In [ ]:
df = pd.read_csv('../data/ratings_v310125.csv')
print(f"Dataset shape: {df.shape}")

# Display basic information about the dataset
df.info()


## 2. Data Preprocessing and Cleaning


In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("\nMissing values per column:")
print(missing_values[missing_values > 0])

# Convert columns to appropriate types if needed
# Convert binary columns to boolean
df['is_healthcare_personnel'] = df['is_healthcare_personnel'].astype(bool)
df['previous_participation'] = df['previous_participation'].astype(bool)

# Display basic statistics
print("\nBasic statistics:")
df.describe()


## 3. Exploratory Data Analysis

### 3.1 Dataset Overview


In [ ]:
# Count unique participants, questions, and answers
unique_participants = df['user_id'].nunique()
unique_questions = df['question_id'].nunique()
unique_answers = df['answer_id'].nunique()

print(f"\nUnique participants: {unique_participants}")
print(f"Unique questions: {unique_questions}")
print(f"Unique answers: {unique_answers}")

# Count LLM vs Human responses
df['source'] = df['source'].str.lower()  # Ensure consistency in source labels
llm_responses = df[df['source'] == 'llm'].shape[0]
human_responses = df[df['source'] == 'human'].shape[0]

print(f"\nLLM responses: {llm_responses}")
print(f"Human responses: {human_responses}")
print(f"Total ratings: {df.shape[0]}")

# Calculate average ratings per user
ratings_per_user = df.groupby('user_id').size()
avg_ratings_per_user = ratings_per_user.mean()
median_ratings_per_user = ratings_per_user.median()

print(f"\nAverage ratings per user: {avg_ratings_per_user:.1f}")
print(f"Median ratings per user: {median_ratings_per_user}")


### 3.2 Participant Demographics


In [ ]:
def generate_demographic_table():
    # Participant groups
    groups = {
        'Diabetes and Healthcare Professional': df[(df['has_diabetes'] == 'yes') & 
                                                 (df['is_healthcare_personnel'] == True)],
        'Only Diabetes': df[(df['has_diabetes'] == 'yes') & 
                           (df['is_healthcare_personnel'] == False)],
        'Only Healthcare Professional': df[(df['has_diabetes'] == 'no') & 
                                         (df['is_healthcare_personnel'] == True)],
        'Neither': df[(df['has_diabetes'] == 'no') & 
                     (df['is_healthcare_personnel'] == False)]
    }
    
    demographic_data = []
    
    # Count unique users in each group
    for group_name, group_df in groups.items():
        unique_users = group_df['user_id'].nunique()
        ratings_count = group_df.shape[0]
        demographic_data.append({
            'Characteristic': 'Participant and Groups',
            'Subgroup': group_name,
            'Frequency (N)': unique_users,
            'Ratings': ratings_count
        })
    
    # Age groups
    age_groups = df.groupby('age_group')
    for age, group in age_groups:
        demographic_data.append({
            'Characteristic': 'Age Group',
            'Subgroup': age,
            'Frequency (N)': group['user_id'].nunique(),
            'Ratings': group.shape[0]
        })
        
    # Gender
    gender_groups = df.groupby('gender')
    for gender, group in gender_groups:
        demographic_data.append({
            'Characteristic': 'Gender',
            'Subgroup': gender,
            'Frequency (N)': group['user_id'].nunique(),
            'Ratings': group.shape[0]
        })
        
    # Education Level
    edu_groups = df.groupby('education_level')
    for edu, group in edu_groups:
        demographic_data.append({
            'Characteristic': 'Education Level',
            'Subgroup': edu,
            'Frequency (N)': group['user_id'].nunique(),
            'Ratings': group.shape[0]
        })
        
    # Healthcare Professional Type
    hcp_groups = df[df['is_healthcare_personnel']].groupby('healthcare_professional_type')
    for hcp_type, group in hcp_groups:
        if pd.notna(hcp_type):  # Skip NaN values
            demographic_data.append({
                'Characteristic': 'Healthcare Professional Type',
                'Subgroup': hcp_type,
                'Frequency (N)': group['user_id'].nunique(),
                'Ratings': group.shape[0]
            })
    
    # Create a DataFrame for better display
    demographic_table = pd.DataFrame(demographic_data)
    
    return demographic_table

# Generate and display the demographic table
demographic_table = generate_demographic_table()
demographic_table


### 3.3 Descriptive Statistics


In [ ]:
def descriptive_stats_by_source():
    metrics = ['knowledge', 'helpfulness', 'empathy']
    stats_data = []
    
    for metric in metrics:
        for source in ['llm', 'human']:
            source_data = df[df['source'] == source][metric]
            
            stats_data.append({
                'Metric': metric.capitalize(),
                'Source': source.upper(),
                'n': len(source_data),
                'Mean': source_data.mean(),
                'SD': source_data.std(),
                '95% CI Lower': source_data.mean() - 1.96 * (source_data.std() / np.sqrt(len(source_data))),
                '95% CI Upper': source_data.mean() + 1.96 * (source_data.std() / np.sqrt(len(source_data))),
                'Median': source_data.median(),
                'Min': source_data.min(),
                'Max': source_data.max()
            })
    
    stats_df = pd.DataFrame(stats_data)
    stats_df['95% CI'] = stats_df.apply(lambda x: f"[{x['95% CI Lower']:.2f}, {x['95% CI Upper']:.2f}]", axis=1)
    
    # Reformat the dataframe to match the paper's table
    formatted_stats = stats_df[['Metric', 'Source', 'n', 'Mean', 'SD', '95% CI', 'Median']]
    formatted_stats['Mean (SD)'] = formatted_stats.apply(lambda x: f"{x['Mean']:.2f} ({x['SD']:.2f})", axis=1)
    
    return formatted_stats[['Metric', 'Source', 'n', 'Mean (SD)', '95% CI', 'Median']]

# Generate descriptive statistics table
desc_stats_table = descriptive_stats_by_source()
desc_stats_table
